## Step 2 - Filtering Key Opinion Leaders Tweets

In this step we are filtering the tweets of the Key Opinion Leaders, that we retrieved in the first step. 


In [3]:
#! pip install inputimeout

In [5]:
import requests
import os
import json
import pandas as pd
import numpy as np
import csv , datetime, unicodedata, time, tweeterid #dateutil.parserm,
import gensim
import openpyxl
import json

from os import path
from datetime import datetime
from collections import Counter
from inputimeout import inputimeout, TimeoutOccurred

data_folder_name = "data_folder"

In [3]:
#! pip install gensim==4.1.2

  Using cached gensim-4.1.2-cp39-cp39-win_amd64.whl (24.0 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\roymad\\Anaconda3\\Lib\\site-packages\\gensim\\_matutils.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
import gensim
gensim.__version__

'4.1.2'

In [4]:
from Brexit_Package import TwitterCrawler

my_token = 'NLP_is_fun' ### Of course one need to provide here a real valid token, unfortunelty we can't publish our token...
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


### Read all the data we have = all the tweets we read from twitter + some Preprocess


In [6]:
import os 

dir_with_all_tweets = os.path.join("KOL tweets")
stop_words_file_name = "stopwords.txt"

score_for_KOP = 0
score_for_key_event = 5
score_for_key_words = 5

key_words = ["brexit", "eu", "deal", "economy"]
threshold_score = 5
verbose = True

KOP_excel_name = "KOP brexit.xlsx"
key_events_excel_name = "Brexit_key_events.xlsx"

stop_words_to_add = ["http", "https", "rt", "co", "vrkhaxde", "oeblog", "rln", "simonjhix", "cctvqfr",
"dcpj", "xvy", "mycekvwlxr", "imydbvvwji", "kkd", "rwp","yfc","fus","tmawgoafhb","edzmidvpel"]

start_fun_time = time.time()
############# step 1 - Reading the tweets data #############
if verbose: print("Step 1: Reading all the csv files")
tweets_tables = []
csv_files_evaluated = []
for root,dirs,files in os.walk(dir_with_all_tweets):
    for file in files:
        if file.endswith(".csv"): #if the file is csv
            csv_files_evaluated.append(file)
            #print(file)
            file_location = os.path.join(dir_with_all_tweets, file)
            tweets_tables.append(pd.read_csv(file_location))

tweets_table = pd.concat(tweets_tables)
tweets_table = tweets_table.drop(labels = ["Unnamed: 0"], axis = 1)
all_tweets_back_up = tweets_table.copy()
if verbose: print("There are: ", tweets_table.shape[0], "Tweets")
    
### removing duplicates rows
tweets_table = tweets_table.drop_duplicates(subset=['author_id_new', 'conv_id_new', "id_new", "created_at", "public_metrics.like_count", "text"])
if verbose: print("There are: ", tweets_table.shape[0], "Non duplicates Tweets")
    
# ############# step 2 - Reading events data #############
# if verbose: print("Step 2: Reading The Events Excel File")
# dir_path_for_events_table = os.path.join(key_events_excel_name)
# events_table = pd.read_excel(dir_path_for_events_table)
# events_table = events_table.drop(labels = ["tweets", "Arguments", "Index"], axis = 1)
# #display(events_table.head())

# ############# step 3 - Reading KOP data #############
# if verbose: print("Step 3: Reading and preprocessing Key Opinion Leaders data")
# dir_path_for_KOP_table = os.path.join(KOP_excel_name)
# KOP_table = pd.read_excel(dir_path_for_KOP_table)
# try:
#     KOP_table.rename(columns = {'Unnamed: 0':'KOP_num',
#                                 "Born In":"Born_in",
#                                 "Twitter acount name" : "twitter_user_name",}, inplace = True)
#     #drop unnecessary columns
#     KOP_table = KOP_table.drop(labels = ["Unnamed: 7", "Source"], axis = 1)
# except: print()

# #remove KOP without twitter account name
# KOP_table = KOP_table[KOP_table['twitter_user_name'].notna()]
# #remove the "@" at the begining of some user names
# KOP_table["clean_user_name"] = KOP_table["twitter_user_name"].apply(lambda x: x.replace("@", ""))

# def take_all_except_first_char(x):
#     try:
#         author_id = tweeterid.handle_to_id(x)
#     except:
#         author_id = "-"
#     return author_id

# KOP_table["author_id"] = KOP_table["clean_user_name"].apply(take_all_except_first_char)

############# step 4-A - Stop-words #############
if verbose: print("Step 4-A: Reading stop words data and adding your stop words")
stopwords_file_name = os.path.join(stop_words_file_name)
stopwords_url = "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords"

if not os.path.isfile(stopwords_file_name):
    stopwords = requests.get(stopwords_url).text.split()
    with open(stopwords_file_name,'w+t', encoding='utf-8') as out_file:
        out_file.write(' '.join(stopwords))
else:
    with open(stopwords_file_name,'rt', encoding='utf-8') as in_file:
        stopwords = in_file.readline().split()
stopwords = set(stopwords)

#### Adding stopWords
for word_i in stop_words_to_add:
    stopwords.add(word_i)

############# step 4-B - contractions #############
if verbose: print("\nStep 4-B: Preprocess the tweets table - the text column - using contractions ")
def clean_text(x):
    from gensim.utils import simple_preprocess
    import contractions
    try:
        x = contractions.fix(x)
        x = ' '.join(simple_preprocess(x))
    except:
        fun = "fun"
    return x
tweets_table['text'] = tweets_table['text'].apply(clean_text)

############# step 5 - Preprocess the tweets table for scoring #############
if verbose: print("\nStep 5: Preprocess the tweets table for scoring:\nRemoving Stop words\
| bigram, trigram, forthgram | merging the tweets table with the KOP and Events tables | ")

start_time = time.time()
#using gensim function to split the text into tokens
tweets_table["text_tokens"] = tweets_table["text"].apply(gensim.utils.simple_preprocess,{"deacc":True, "min_len":2,"max_len":25})

#remove stopwords:
def remove_stopwords(tokens, stopwords):
    return [token for token in tokens if token not in stopwords]
tweets_table['text_tokens'] = tweets_table['text_tokens'].apply(remove_stopwords, stopwords=stopwords)

################################## add bygrams to the tokens
#print(f"connector words: {gensim.models.phrases.ENGLISH_CONNECTOR_WORDS}")
## train bygram model
bigram = gensim.models.Phrases(tweets_table['text_tokens'], min_count=7, threshold=2, connector_words=gensim.models.phrases.ENGLISH_CONNECTOR_WORDS)
bigram_model = gensim.models.phrases.Phraser(bigram)
if verbose: print(f"found {len(bigram_model.phrasegrams)} bigrams")
# bigram_model.phrasegrams #the bygrams the model found
tweets_table['text_tokens'] = tweets_table['text_tokens'].apply(lambda x: bigram_model[x])
###### add trigram to the tokens
trigram = gensim.models.Phrases(tweets_table['text_tokens'], min_count=7, threshold=2, connector_words=gensim.models.phrases.ENGLISH_CONNECTOR_WORDS)
trigram_model = gensim.models.phrases.Phraser(trigram)
if verbose: print(f"found {len(trigram_model.phrasegrams)} trigram")
#trigram_model.phrasegrams #the trigram that the model found
tweets_table['text_tokens'] = tweets_table['text_tokens'].apply(lambda x: trigram_model[x])
############################################################################
###### add forthgram to the tokens
forthgram = gensim.models.Phrases(tweets_table['text_tokens'], min_count=7, threshold=6, connector_words=gensim.models.phrases.ENGLISH_CONNECTOR_WORDS)
forthgram_model = gensim.models.phrases.Phraser(forthgram)
if verbose: print(f"found {len(forthgram_model.phrasegrams)} forthgram")
#forthgram_model.phrasegrams #the trigram that the model found
tweets_table['text_tokens'] = tweets_table['text_tokens'].apply(lambda x: forthgram_model[x])
############################################################################
### add tweet date column
def take_only_10_first_char(x):
    return(x[0:10])
tweets_table["created_at_date"] = tweets_table["created_at"].apply(take_only_10_first_char)
tweets_table["created_at_date"] = pd.to_datetime(tweets_table["created_at_date"])

tweets_table["Year_tweet"] = pd.DatetimeIndex(tweets_table['created_at']).year

### add column is_in_special_date that checks wheter the tweet was published in a special event day
#tweets_table['is_in_special_date'] = tweets_table['created_at_date'].apply(lambda x : pd.Series(x).isin(events_table["Date"]).any())

# ### Joining the event table so we can get additional information for the special events
# tweets_table = tweets_table.merge(events_table, left_on = "created_at_date", right_on = "Date", how = "left")
# tweets_table.rename(columns = {'Date':'Event_Date'}, inplace = True)
# tweets_table['Event_Date'] = tweets_table['Event_Date'].fillna("No-Event")

# ### Joining the KOP table so we can get additional information for the KOP
# tweets_table = tweets_table.merge(KOP_table, left_on = "author_id", right_on = "author_id", how = "left")
# try:
#     tweets_table['KOP_num'] = tweets_table['KOP_num'].fillna("No-KOP")
#     tweets_table['Index'] = tweets_table['Index'].fillna("No-KOP")
#     tweets_table['Name'] = tweets_table['Name'].fillna("No-KOP")
#     tweets_table['Born_in'] = tweets_table['Born_in'].fillna("No-KOP")
#     tweets_table['Role'] = tweets_table['Role'].fillna("No-KOP")
#     tweets_table['Place'] = tweets_table['Place'].fillna("No-KOP")
# except: print()

if verbose: print("\nFinish preprocessing the tweets table, it took:", round(time.time() - start_time,3), "seconds")

# ############# step 6 - Scoring #############
# if verbose: print("Step 6: Scoring the Tweets")

# KOP_ids = list(set(KOP_table.author_id))

# ### the following scorer - count the number of times the key words apprear in a certain text. If a certain word (brexit)
# # appear more than once, it will be counted twice
# def scorer_keywords_brexit(tweet_tokens, key_words):
#     #scoring the keywords:
#     count_key_words = 0
#     for word in key_words:
#         count_key_words += tweet_tokens.count(word)
#     return count_key_words
# ##########################################################################################################
# def scorer_KOP_brexit(author_id, KOP_ids):
#     if author_id in KOP_ids:
#         score = 1
#     else:
#         score = 0
#     return score
# ##########################################################################################################
# ### adding year column
# #tweets_table["Year_tweet"] = pd.DatetimeIndex(tweets_table['created_at']).year

# ### scoring key-words
# tweets_table["score_key_words"] = tweets_table['text_tokens'].apply(scorer_keywords_brexit, key_words = key_words)

# #scoring KOP: if the author is KOP then the score in this column will be 1, else 0
# tweets_table["score_KOP"] = tweets_table['author_id'].apply(scorer_KOP_brexit, KOP_ids = KOP_ids)

# ### scoring events - column is_in_special_date

# #tweets_table["total_score"]: adding the weights of each scorer
# tweets_table["total_score"] = score_for_key_words*tweets_table['score_key_words'] + score_for_key_event*tweets_table["is_in_special_date"] + score_for_KOP*tweets_table["score_KOP"]

# #Geting a table with all the tweets that passed the score threshold
# tweets_table_filtered = tweets_table[tweets_table["total_score"]>=threshold_score].copy()
# #Sort the filtered table by score such that the tweets with the highest score will be first
# tweets_table_filtered.sort_values(by = "total_score", ascending=False, inplace=True)

print("\nFinish!\nTotal time:", round(time.time() - start_fun_time,3),
    "Seconds (",round((time.time() - start_fun_time)/60,3), "Minutes)")

bigrams_models = [bigram, trigram, forthgram]
#return (tweets_table, csv_files_evaluated, bigrams_models)

Step 1: Reading all the csv files


C:\Users\roymad\AppData\Local\Temp\ipykernel_39372\2557764432.py:30: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_tables.append(pd.read_csv(file_location))
C:\Users\roymad\AppData\Local\Temp\ipykernel_39372\2557764432.py:30: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_tables.append(pd.read_csv(file_location))


There are:  1362559 Tweets
There are:  979821 Non duplicates Tweets
Step 4-A: Reading stop words data and adding your stop words

Step 4-B: Preprocess the tweets table - the text column - using contractions 

Step 5: Preprocess the tweets table for scoring:
Removing Stop words| bigram, trigram, forthgram | merging the tweets table with the KOP and Events tables | 
found 95256 bigrams
found 109745 trigram
found 87388 forthgram

Finish preprocessing the tweets table, it took: 142.882 seconds

Finish!
Total time: 200.017 Seconds ( 3.334 Minutes)


In [7]:
print(f"Total number of tweets={tweets_table.shape[0]}")


Total number of tweets=979821


## Writing the filtered table to csv

In [ ]:
tweets_table.to_csv(os.path.join(data_folder_name, "tweets_table_all.csv"))

# Not relevant:

### Add basic sentiment using NLTK

In [54]:
import nltk
nltk.download('vader_lexicon')

def nltk_sentiment(row, sentiment_intensity_analyzer, column_text_name = "text"):
    import numpy as np, pandas as pd
    sentiment_names = ['negative', 'neutral', 'positive']
    nltk_sentiment = sentiment_intensity_analyzer.polarity_scores(row[column_text_name])
    
    #nltk_sentiment['nltk_verdict'] = sentiment_names[np.argmax([nltk_sentiment[s] for s in ['neg','neu','pos']])]
    if nltk_sentiment['neu']>0.95:
        nltk_sentiment['nltk_verdict'] = 'neutral'
    elif nltk_sentiment['neg'] < nltk_sentiment['pos']:
        nltk_sentiment['nltk_verdict'] = 'positive'
    else:
        nltk_sentiment['nltk_verdict'] = 'negative'
    #nltk_sentiment['ground_truth'] = row['airline_sentiment']
    #nltk_sentiment['ground_truth_value'] = {'negative': -1, 'neutral':0, 'positive':1 }[nltk_sentiment['ground_truth']]
    return pd.Series(nltk_sentiment)



from nltk.sentiment import SentimentIntensityAnalyzer
sentiment_intensity_analyzer = SentimentIntensityAnalyzer()

nltk_sentiment_tweets_table = tweets_table.apply(nltk_sentiment, sentiment_intensity_analyzer=sentiment_intensity_analyzer, column_text_name = "text"  ,axis=1)

##### Adding the sentiment analysis columns
tweets_table["KOL_Tweet_negative"] = nltk_sentiment_tweets_table.neg
tweets_table["KOL_Tweet_neutural"] = nltk_sentiment_tweets_table.neu
tweets_table["KOL_Tweet_positive"] = nltk_sentiment_tweets_table.pos
tweets_table["KOL_Tweet_compound"] = nltk_sentiment_tweets_table.compound

#### This columns contain the final "verdict" of the NLTK Model
tweets_table["KOL_Tweet_nltk_verdict"] = nltk_sentiment_tweets_table.nltk_verdict




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Roy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
tweets_table.head()

### Writing the filtered table to excel

In [18]:
#tweets_table.to_excel(os.path.join(data_folder_name, "tweets_table_all3.xlsx"))


'data_folder'

In [8]:
### write to csv
tweets_table.to_csv(os.path.join(data_folder_name, "tweets_table_all.csv"))

In [31]:
# tweets_table_filtered.to_excel(os.path.join(data_folder_name, "tweets_table_filtered2.xlsx"))

## Add language classification
And filter the non English Tweets

In [ ]:

language_classifications = []
for row in tqdm_notebook(tweets_table.text): #tqdm_notebook - to pring progress bar
    language_classification, score = langid.classify(row)
    language_classifications.append(language_classification)


tweets_table["language"] = language_classifications
tweets_table = tweets_table[tweets_table["language"] == "en"]

### reset index
tweets_table.reset_index(inplace=True)
tweets_table["index"] = tweets_table.index

print(f"Total number of tweets of the KOL in English={tweets_table.shape[0]}")


In [55]:
### Get the number of tweets per KOL
tweets_table[["author_id_new", "id", "users.name", "Year_tweet"]].groupby(by = ["users.name", "Year_tweet"]).count().sort_values(by = "id", ascending = False)

author_id_new     id
users.name                 Year_tweet                      
Jeremy Vine                2019                19406  19406
ChukaUmunna                2019                 8512   8512
Sadiq Khan                 2019                 7318   7318
Jeremy Vine                2018                 7171   7171
                           2016                 7118   7118
...                                              ...    ...
Rupert Murdoch             2017                    1      1
Rt. Hon. Michael Heseltine 2015                    1      1
Paul Dacre                 2016                    1      1
                           2014                    1      1
Olly Robbins               2018                    1      1

[326 rows x 2 columns]

In [58]:
### Get the number of tweets per KOL
tweets_table_filtered[["author_id_new", "id", "users.name", "Year_tweet"]].groupby(by = ["users.name", "Year_tweet"]).count().sort_values(by = "id", ascending = False)

author_id_new   id
users.name      Year_tweet                    
Charles Grant   2017                  869  869
                2016                  852  852
Martin Selmayr  2017                  776  776
ChukaUmunna     2017                  709  709
Charles Grant   2019                  684  684
...                                   ...  ...
Dr Liam Fox MP  2014                    1    1
Steve O'Connell 2014                    1    1
Dyson           2018                    1    1
Matteo Renzi    2014                    1    1
Rupert Murdoch  2017                    1    1

[294 rows x 2 columns]

In [14]:
tweets_table[["author_id_new", "id", "users.name", "Year_tweet"]].groupby(by = ["Year_tweet"]).count().sort_values(by = "id", ascending = False)

,author_id_new,id,users.name
Year_tweet,,,
2019,127945,127945,127945
2017,68602,68602,68602
2016,67645,67645,67645
2018,61387,61387,61387
2015,56626,56626,56626
2014,45455,45455,45455
2020,8090,8090,8090
